***Objective: build a naive bayes classifier to perform document classification.***

# Preprocessing

In [ ]:
# Load important library
import numpy as np
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize

# confusion matrix
from sklearn.metrics import confusion_matrix

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Load dataset
df = pd.read_csv('CS173-published-sheet - Sheet1.csv')
# View first few columns
df.head()

,Sadness Lexicons,Sadness Sentences,Joy Lexicons,Joy Sentences,Fear Lexicons,Fear Sentences,Anger Lexicons,Anger Sentences,Surprise Lexicons,Surprise Sentence,Disgust Lexicons,Disgust Sentences,Sadness + Joy Lexicons,Sadness + Joy Sentences,Fear + Anger Lexicons,Fear + Anger Sentences,Surprise + Disgust Lexicons,Surprise + Disgust Sentences,Sadness + Joy + Fear Lexicons,Sadness + Joy + Fear Sentences
0,"abduction, devastate",The devastating news of the child's abduction ...,"child, laughter, sunny",It was a sunny summer morning and the laughter...,shiver,As he walked in the dead of night he could hea...,reckless,While driving his family to a restaurant a car...,"startle, unexpected",She was startled unexpectedly as everyone spra...,trash,She had forgotten to take out the trash before...,"melancholy, pleasant","When I visited my old childhood home, I felt a...","fearful, harm, provoking, rage",Tom was fearful of his bully and remained doci...,"revolting, shock",He opened the package of his food delivery and...,"anxious, proud, sorrowful",The parents watched their son leave for colleg...
1,"wretched, wrongly",The wretched people have chosen wrongly.,"youth, zeal",The youth are filled with zeal.,"abandoned, yell, youth",The abandoned youth could be heard yelling.,"abolish, wrongful",We must abolish it for its wrongful acts.,yelp,He let out a yelp when his friend jumped out f...,abhorrent,The mysterious slime emitted an abhorrent smell.,"abysmal, advance","They have advanced to the next round, despite ...","animosity, agitation",His animosity was made worse by the repeated\...,"bewildered, appalling",She was bewildered by their appalling behavior.,"apologize, admirable, aggression, afraid",After seeing the group’s admirable performance...
2,"Stifle, Mausoleum","She was not fond of graveyards, let alone the\...","Fidelity, Praiseworthy",Ms. Smith taught the lesson to fidelity. The o...,"\r\nAbyss, Cripple","Leaning over the edge, I peered into the abyss...","Mock, Violence",Your actions make a mockery of ethics! I must ...,"Jolt, Shatter",The sound of my alarms jolts me awake. It shat...,"Unnatural, Plague, Grotesque","His illness left his skin unnatural, like plas...","Winning, Closure","Despite finally winning, I feel robbed of clos...","Abomination, Monstrosity, Affront","What you have created is an abomination, a bon...","Rave, Betray, Dreadfully",Enough of your dreadful raving... Did you forg...,"Endless, Destination, Retirement, Weight",So begins an endless journey with no destinati...
3,"abandon,abandoned,abandonment,abduction,aborti...",I’m feeling very anxious about going back to w...,"absolution, abundance,abundant,accolade,accomp...",The women’s soccer team was proud that they ac...,"abandon,abandoned,abandonment,abduction,abhor,...",I’m afraid we might get trapped in the cave si...,"abandoned,abandonment,abhor,abhorrent,abolish,...","After getting yelled at for 30 minutes, Sam be...","underestimate,undiscovered,unexpected,unexpect...",I underestimated how urgent the call was.,"aberration,abhor,abhorrent,abject,abnormal,abo...",I can’t believe how wasteful the food industry...,"true,twinkle,unbeaten,unconstrained,undying,un...",Although my heart aches because I will never b...,"vermin,versus,veto,vicious,victim,victimized,v...",I hate Tony for wrongfully accusing me of stea...,"underestimate,undiscovered,unexpected,unexpect...",When we looked under the couch we found the un...,"wail,wallow,wan,wane,wanting,warfare,warp,wast...",She starts to weep because she knows that I mi...
4,lonely,Sometimes I feel like something is wrong with ...,"engaged, rekindle",It's crazy to see that Jane and Mike were able...,"onfined, tumor, scare",Ever since the doctors found a tumor in Jessie...,"grudge, incompetent",I cannot believe that you’re still holding a g...,"celebrity, aspiration",It’s always been an aspiration of mine to be a...,nasty,I think it’s so nasty that he’s a grown man an...,"tough, closure, glad",It must have been tough 

In [ ]:
# Split the dataframe
# First 30 rows as training dataset
df_train = df.iloc[:30]

# Next 10 rows as validation dataset
# It seems that in this HW, validation dataset is not really used
# Because in our case, the testing dataset and validation dataset kind of the same thing
# They are both testing dataset with known label
df_validation = df.iloc[30:40]
# Drop index
df_validation.reset_index(drop=True, inplace=True)

# Next ~10 rows as testing dataset
df_test = df.iloc[40:]
# Drop index
df_test.reset_index(drop=True, inplace=True)

In [ ]:
# Create a list of labels
labels = ["Fear", "Anger", "Surprise", "Disgust", "Sadness", "Joy"]

## Filter Sentences

In [ ]:
# Filter the training dataset
# Only leave columns that contains "Sentences"
df_train_Sen = df_train.filter(like="Sentences")
# View first couple rows
df_train_Sen.head()

,Sadness Sentences,Joy Sentences,Fear Sentences,Anger Sentences,Disgust Sentences,Sadness + Joy Sentences,Fear + Anger Sentences,Surprise + Disgust Sentences,Sadness + Joy + Fear Sentences
0,The devastating news of the child's abduction ...,It was a sunny summer morning and the laughter...,As he walked in the dead of night he could hea...,While driving his family to a restaurant a car...,She had forgotten to take out the trash before...,"When I visited my old childhood home, I felt a...",Tom was fearful of his bully and remained doci...,He opened the package of his food delivery and...,The parents watched their son leave for colleg...
1,The wretched people have chosen wrongly.,The youth are filled with zeal.,The abandoned youth could be heard yelling.,We must abolish it for its wrongful acts.,The mysterious slime emitted an abhorrent smell.,"They have advanced to the next round, despite ...",His animosity was made worse by the repeated\...,She was bewildered by their appalling behavior.,After seeing the group’s admirable performance...
2,"She was not fond of graveyards, let alone the\...",Ms. Smith taught the lesson to fidelity. The o...,"Leaning over the edge, I peered into the abyss...",Your actions make a mockery of ethics! I must ...,"His illness left his skin unnatural, like plas...","Despite finally winning, I feel robbed of clos...","What you have created is an abomination, a bon...",Enough of your dreadful raving... Did you forg...,So begins an endless journey with no destinati...
3,I’m feeling very anxious about going back to w...,The women’s soccer team was proud that they ac...,I’m afraid we might get trapped in the cave si...,"After getting yelled at for 30 minutes, Sam be...",I can’t believe how wasteful the food industry...,Although my heart aches because I will never b...,I hate Tony for wrongfully accusing me of stea...,When we looked under the couch we found the un...,She starts to weep because she knows that I mi...
4,Sometimes I feel like something is wrong with ...,It's crazy to see that Jane and Mike were able...,Ever since the doctors found a tumor in Jessie...,I cannot believe that you’re still holding a g...,I think it’s so nasty that he’s a grown man an...,It must have been tough to find out all the de...,It’s insane how Daniel was able to act normal ...,It’s concerning how Pearson isn’t in much dist...,It really makes you wonder what people are cap...


In [ ]:
# Create a list to store sentence with its corresponding label
Dictionary_Sen = []
# Loop though each column
for column in df_train_Sen.columns:
  # Loop through each label and see if that label is mentioned in the current column
  # This way, if sentence has multiple label, it will be added multiple times as well
  for label in labels:
    if label in column:
      # Loop through every sentence
      # There is NaN, so we need dropna()
      for sentence in df_train_Sen[column].dropna():
        # Append the sentence label pair
        Dictionary_Sen.append([sentence, label])

In [ ]:
# Convert the list to a pandas dataframe
Dictionary_Sen_train = pd.DataFrame(Dictionary_Sen, columns=["Sentence", "Label"])
Dictionary_Sen = pd.DataFrame(Dictionary_Sen, columns=["Sentence", "Label"])
Dictionary_Sen

,Sentence,Label
0,The devastating news of the child's abduction ...,Sadness
1,The wretched people have chosen wrongly.,Sadness
2,"She was not fond of graveyards, let alone the\...",Sadness
3,I’m feeling very anxious about going back to w...,Sadness
4,Sometimes I feel like something is wrong with ...,Sadness
...,...,...
408,Although I am greatly afflicted by the incomin...,Joy
409,"“The couple traveled to a dreamy destination,\...",Joy
410,"I was totally distraught when I lost my cat, b...",Joy
411,"She felt a mix of emotions, feeling exhilarati...",Joy


In [ ]:
# Tokenize each sentence
Dictionary_Sen_train["Sentence"] = Dictionary_Sen_train["Sentence"].apply(word_tokenize)

In [ ]:
# For each label
# Output 1 example of tokenized sentence
for label in labels:
  print(label)
  print(Dictionary_Sen_train[Dictionary_Sen["Label"] == label]["Sentence"].iloc[0])

Fear
['As', 'he', 'walked', 'in', 'the', 'dead', 'of', 'night', 'he', 'could', 'hear', 'sudden', 'footsteps', 'echoing', 'from', 'the', 'alleyway', ',', 'a', 'shiver', 'went', 'down', 'his', 'spine', '.']
Anger
['While', 'driving', 'his', 'family', 'to', 'a', 'restaurant', 'a', 'car', 'recklessly', 'changed', 'lanes', 'barely', 'missing', 'him', ',', 'his', 'face', 'flushed', 'red', 'with', 'fury', 'knowing', 'how', 'close', 'his', 'family', 'was', 'to', 'being', 'seriously', 'harmed', '.']
Surprise
['He', 'opened', 'the', 'package', 'of', 'his', 'food', 'delivery', 'and', 'was', 'shocked', 'to', 'see', 'a', 'dead', 'insect', 'laying', 'on', 'top', 'his', 'food', ',', 'the', 'idea', 'of', 'accidentally', 'eating', 'it', 'revolted', 'him', '.']
Disgust
['She', 'had', 'forgotten', 'to', 'take', 'out', 'the', 'trash', 'before', 'leaving', 'for', 'holiday', ',', 'when', 'she', 'returned', 'she', 'was', 'repulsed', 'by', 'the', 'stench', 'of', 'decaying', 'food', '.']
Sadness
['The', 'devas

## Filter Word Tokens/Lexicons

There may be two ways to construct the word count dictionary. \
We either tokenize the sentence first, get a English stopword list from nltk library, and for each sentence, remove stopwords from that sentence and count frequency for rest of the words. \
But the previous approach does not remove words that are frequent among all labels, it simply remove stopwords as its best guess. Here in the dataset we can see it already has marked lexicons under each label, and we could simply build the dictionary based on these lexicons and achieve a similar result, as they are word highly related to its corresponding label.

In the code afterwards, I chose to build the dictionary base on Lexicons.

In [ ]:
# Filter the training dataset
# Only leave columns that contains "Sentences"
df_train_Lex = df_train.filter(like="Lexicons")
# View first couple rows
df_train_Lex.head()

,Sadness Lexicons,Joy Lexicons,Fear Lexicons,Anger Lexicons,Surprise Lexicons,Disgust Lexicons,Sadness + Joy Lexicons,Fear + Anger Lexicons,Surprise + Disgust Lexicons,Sadness + Joy + Fear Lexicons
0,"abduction, devastate","child, laughter, sunny",shiver,reckless,"startle, unexpected",trash,"melancholy, pleasant","fearful, harm, provoking, rage","revolting, shock","anxious, proud, sorrowful"
1,"wretched, wrongly","youth, zeal","abandoned, yell, youth","abolish, wrongful",yelp,abhorrent,"abysmal, advance","animosity, agitation","bewildered, appalling","apologize, admirable, aggression, afraid"
2,"Stifle, Mausoleum","Fidelity, Praiseworthy","\r\nAbyss, Cripple","Mock, Violence","Jolt, Shatter","Unnatural, Plague, Grotesque","Winning, Closure","Abomination, Monstrosity, Affront","Rave, Betray, Dreadfully","Endless, Destination, Retirement, Weight"
3,"abandon,abandoned,abandonment,abduction,aborti...","absolution, abundance,abundant,accolade,accomp...","abandon,abandoned,abandonment,abduction,abhor,...","abandoned,abandonment,abhor,abhorrent,abolish,...","underestimate,undiscovered,unexpected,unexpect...","aberration,abhor,abhorrent,abject,abnormal,abo...","true,twinkle,unbeaten,unconstrained,undying,un...","vermin,versus,veto,vicious,victim,victimized,v...","underestimate,undiscovered,unexpected,unexpect...","wail,wallow,wan,wane,wanting,warfare,warp,wast..."
4,lonely,"engaged, rekindle","onfined, tumor, scare","grudge, incompetent","celebrity, aspiration",nasty,"tough, closure, glad","insane, inhumane","catastrophe, lawsuit, distress, wretched","abandoned, youth, brutal, abuse"


In [ ]:
# Create a Lexicon count for each label
# First create a empty dictionary for each label
Dictionary_Lex = {}
for label in labels:
  Dictionary_Lex[label] = {}

# Loop though each column
for column in df_train_Lex.columns:
  # Loop through each label and see if that label is mentioned in the current column
  # This way, if lexicon has multiple label, it will be added multiple times as well
  for label in labels:
    if label in column:
      # Loop through every sentence
      # There is NaN, so we need dropna()
      for lexicons in df_train_Lex[column].dropna():
        # Preprocessing the lexicons, by removing special characters
        lexicons = lexicons.replace("(","")
        lexicons = lexicons.replace(")","")
        lexicons = lexicons.replace(".","")
        # There are two styles of "," and ", "
        # We will first change all ", " to ","
        lexicons = lexicons.replace(", ",",")
        # Then we will remove ",", replace it with " "
        lexicon_list = lexicons.replace(","," ").split()
        # Loop through each word in the lexicon list
        for lexicon in lexicon_list:
          # If the word is not in the dictionary, add it by setting it to 1
          if lexicon not in Dictionary_Lex[label]:
            Dictionary_Lex[label][lexicon] = 1
          # If the word is in the dictionary already, + 1
          else:
            Dictionary_Lex[label][lexicon] += 1

In [ ]:
# Convert the Dictionary to a pandas dataframe
Dictionary_Lex = pd.DataFrame(Dictionary_Lex)
# Replace all NaN with 0
Dictionary_Lex = Dictionary_Lex.fillna(0)
# View first couple rows
# The Dictionary is case sensitive btw
Dictionary_Lex.head()

,Fear,Anger,Surprise,Disgust,Sadness,Joy
shiver,1.0,0.0,0.0,0.0,0.0,0.0
abandoned,4.0,2.0,0.0,0.0,6.0,4.0
yell,6.0,4.0,1.0,0.0,1.0,1.0
youth,7.0,2.0,1.0,0.0,5.0,6.0
Abyss,2.0,0.0,0.0,0.0,0.0,0.0


# Naive Bayes

In [ ]:
# Compute the priors of each label
# Basically we count number of sentence with label / total sentence
total_number = len(Dictionary_Sen_train)
# Create a key-value dictionary to store the prior probability for each label
Priors = {}
for label in labels:
  Priors[label] = len(Dictionary_Sen_train[Dictionary_Sen_train["Label"] == label]) / total_number
  print("P(" + label + ") =", Priors[label])

# Verification step
sum = 0
for label in labels:
  sum += Priors[label]
print("Sum of Priors =", sum)

P(Fear) = 0.21549636803874092
P(Anger) = 0.14527845036319612
P(Surprise) = 0.06779661016949153
P(Disgust) = 0.14043583535108958
P(Sadness) = 0.21549636803874092
P(Joy) = 0.21549636803874092
Sum of Priors = 1.0


## Calculate the likelihoods of each word given an emotion category using Laplace smoothing

In [ ]:
# We can get the entire vocabulary list from Dictionary's index
vocab_list = Dictionary_Lex.index.tolist()
# Length of unique words is just the length of the vocab list
vocab_length = len(vocab_list)
vocab_length

941

In [ ]:
# Count number of words (not unique words) under each label
Label_word_count = {}
for label in labels:
  # Sum of lexicons count under each label
  Label_word_count[label] = Dictionary_Lex[label].sum()
  print("Number of words under " + label + " =", Label_word_count[label])

Number of words under Fear = 601.0
Number of words under Anger = 335.0
Number of words under Surprise = 247.0
Number of words under Disgust = 242.0
Number of words under Sadness = 590.0
Number of words under Joy = 605.0


In [ ]:
# Calculate the likelihood for each word in the vocab list under each label
Likelihood = {}

# Loop through Dictionary_Lex's indexes
# Where we store the vocab there
for word in Dictionary_Lex.index:
  Likelihood[word] = {}
  # Loop through each label
  for label in labels:
    # Get the current word count under current label
    curr_word_count = Dictionary_Lex.at[word, label]
    # Apply laplace smoothing
    Likelihood[word][label] = (curr_word_count + 1) / (Label_word_count[label] + vocab_length)

We do not need to convert to Pandas Dataframe, this map should be good enough for further processing

## Implement the Naïve Bayes classifier to predict the emotion of a given text

In [ ]:
from os import pread
def Naive_Bayes_Sentence(sentence):
  # Tokenize the sentence first
  sentence = word_tokenize(sentence)

  # Get a valid word list (word that appear in the vocab_list)
  valid_word = []
  for word in sentence:
    if word in vocab_list:
      valid_word.append(word)
  print("valid lexicons: ", valid_word)

  # Parameters to store max likelihood, and predicting label
  predict_label = ""
  max_likelihood = -float("inf")
  # Calculate the Max Likelihood for the sentence under each label
  for label in labels:
    # Initial probability
    probability = 1
    # Multiply by the prior
    probability *= Priors[label]
    for word in valid_word:
      probability *= Likelihood[word][label]
    print("P(" + label + "|s) =", probability)
    # Check if current label has high probability
    if probability > max_likelihood:
      max_likelihood = probability
      predict_label = label

  # Output the predicted label
  print("Predicted label =", predict_label)
  return predict_label

In [ ]:
# Given input text
s = "As she hugged her daughter goodbye on the first day of college, she felt both sad to see her go and joyful knowing that she was embarking on a new and exciting chapter in her life."

In [ ]:
Naive_Bayes_Sentence(s)

valid lexicons:  ['daughter']
P(Fear|s) = 0.00013975121143887219
P(Anger|s) = 0.00011385458492413488
P(Surprise|s) = 5.706785367802317e-05
P(Disgust|s) = 0.00011871161060954318
P(Sadness|s) = 0.00014075530244202542
P(Joy|s) = 0.00027877926007599085
Predicted label = Joy


'Joy'

# Evaluating the Classifier on Test Set

In [ ]:
# We first need to create a dataframe for {sentence, label}
# Will later add a column for {predicted_label}
df_test_Sen = df_test.filter(like="Sentences")

In [ ]:
# Create a list to store sentence with its corresponding label
Dictionary_Sen_test = []
# Loop though each column
for column in df_test_Sen.columns:
  # Loop through each label and see if that label is mentioned in the current column
  # This way, if sentence has multiple label, it will be added multiple times as well
  for label in labels:
    if label in column:
      # Loop through every sentence
      # There is NaN, so we need dropna()
      for sentence in df_test_Sen[column].dropna():
        # Append the sentence label pair
        Dictionary_Sen_test.append([sentence, label])

In [ ]:
# Convert the list to a pandas dataframe
# Label here refers to true label
Dictionary_Sen_test = pd.DataFrame(Dictionary_Sen_test, columns=["Sentence", "Label"])
Dictionary_Sen_test

,Sentence,Label
0,"He sat alone by the window, lost in a pensive ...",Sadness
1,Tim was unlucky. He had a traumatic experience...,Sadness
2,"After his wife left him, he lost his job, and ...",Sadness
3,"The news caused my world to collapse, leaving ...",Sadness
4,He was distraught by the horrors he saw during...,Sadness
...,...,...
116,Link’s quest to save Hyrule from calamity Gann...,Joy
117,Though he was rejoicing that the stab wound lo...,Joy
118,I worry that someday I’ll forget the love I’ve...,Joy
119,She yelled urgently before I was able to leave...,Joy


In [ ]:
# Now we will loop through the dataframe
# For each sentence, we will apply the Naive Bayes algorithm and generate a predicted label
Dictionary_Sen_test["Predicted_Label"] = Dictionary_Sen_test["Sentence"].apply(Naive_Bayes_Sentence)

valid lexicons:  []
P(Fear|s) = 0.21549636803874092
P(Anger|s) = 0.14527845036319612
P(Surprise|s) = 0.06779661016949153
P(Disgust|s) = 0.14043583535108958
P(Sadness|s) = 0.21549636803874092
P(Joy|s) = 0.21549636803874092
Predicted label = Fear
valid lexicons:  ['traumatic', 'horrific', 'isolation']
P(Fear|s) = 1.1754842906097677e-10
P(Anger|s) = 1.3985537794947827e-10
P(Surprise|s) = 4.043511268551336e-11
P(Disgust|s) = 8.482496869181763e-11
P(Sadness|s) = 1.20100379180392e-10
P(Joy|s) = 5.83191903815528e-11
Predicted label = Anger
valid lexicons:  ['anguish']
P(Fear|s) = 0.00013975121143887219
P(Anger|s) = 0.00022770916984826976
P(Surprise|s) = 5.706785367802317e-05
P(Disgust|s) = 0.00011871161060954318
P(Sadness|s) = 0.0005630212097681017
P(Joy|s) = 0.00027877926007599085
Predicted label = Sadness
valid lexicons:  ['feeling']
P(Fear|s) = 0.00027950242287774437
P(Anger|s) = 0.00011385458492413488
P(Surprise|s) = 5.706785367802317e-05
P(Disgust|s) = 0.00011871161060954318
P(Sadness|s)

In [ ]:
# Peak look
Dictionary_Sen_test.head()

,Sentence,Label,Predicted_Label
0,"He sat alone by the window, lost in a pensive ...",Sadness,Fear
1,Tim was unlucky. He had a traumatic experience...,Sadness,Anger
2,"After his wife left him, he lost his job, and ...",Sadness,Sadness
3,"The news caused my world to collapse, leaving ...",Sadness,Joy
4,He was distraught by the horrors he saw during...,Sadness,Fear


## Confusion Matrix

In [ ]:
# Now we can generate the confusion matrix since we have True label and Predicted Label for all sentence
# Generate a 6 x 6 confusion matrix
# reference: https://scikit-learn.org/1.5/modules/generated/sklearn.metrics.confusion_matrix.html
y_true = Dictionary_Sen_test["Label"]
y_pred = Dictionary_Sen_test["Predicted_Label"]
confusion_matrix(y_true, y_pred, labels=labels)

array([[11,  2,  0,  2,  7,  4],
       [10,  3,  0,  1,  3,  1],
       [ 1,  0,  0,  5,  2,  0],
       [ 3,  0,  0, 11,  3,  0],
       [ 4,  2,  0,  0, 10, 10],
       [ 3,  0,  0,  0, 11, 12]])

In [ ]:
# We can convert confusion matrix to pandas dataframe for better readability
# Set labels as both column and index
confusion_matrix_df = pd.DataFrame(confusion_matrix(y_true, y_pred, labels=labels), index=labels, columns=labels)
confusion_matrix_df

,Fear,Anger,Surprise,Disgust,Sadness,Joy
Fear,11,2,0,2,7,4
Anger,10,3,0,1,3,1
Surprise,1,0,0,5,2,0
Disgust,3,0,0,11,3,0
Sadness,4,2,0,0,10,10
Joy,3,0,0,0,11,12


## Calculate accuracy, precision, recall, and F1-score

In [ ]:
# Find number of True Negative for the given label
# Bascially sum of all number that row or column is not "label"
def find_TN(label):
  TN = 0
  for label_i in labels:
    for label_j in labels:
      # Column and index in same order anyways
      if label_i != label and label_j != label:
        TN += confusion_matrix_df.at[label_i, label_j]
  return TN

In [ ]:
# Calculate accuracy, precision, recall, and F1-score
def calculate_performance(label):
  # Find total number of sentence
  total_sentence_number, _ = Dictionary_Sen_test.shape

  # Find accuracy of the given label based on confusion matrix
  # TP + TN / total sentence number
  TN = find_TN(label)
  TP = confusion_matrix_df.at[label, label]
  accuracy = (TP + TN) / total_sentence_number

  # Find precision of the given label based on confusion matrix
  # TP/TP + FP
  FP = 0
  for label_j in labels:
    if label_j != label:
      # Same row, exclude when column == label
      FP += confusion_matrix_df.at[label, label_j]
  precision = TP / (TP + FP)

  # Find recall of the given label based on confusion matrix
  # TP/TP + FN
  FN = 0
  for label_i in labels:
    if label_i != label:
      # Same column, exclude when row == label
      FN += confusion_matrix_df.at[label_i, label]
  recall = TP / (TP + FN)

  # Find f1_score of the given label based on confusion matrix
  f1_score = (2 * precision * recall) / (precision + recall)
  return accuracy, precision, recall, f1_score

In [ ]:
accuracy, precision, recall, f1_score = calculate_performance("Joy")
print("Accuracy =", accuracy)
print("Precision =", precision)
print("Recall =", recall)
print("F1-score =", f1_score)

Accuracy = 0.7603305785123967
Precision = 0.46153846153846156
Recall = 0.4444444444444444
F1-score = 0.4528301886792453


## Use existing Naive Bayes implementation

In [ ]:
# Existing testing dataset
Dictionary_Sen_test

,Sentence,Label,Predicted_Label
0,"He sat alone by the window, lost in a pensive ...",Sadness,Fear
1,Tim was unlucky. He had a traumatic experience...,Sadness,Anger
2,"After his wife left him, he lost his job, and ...",Sadness,Sadness
3,"The news caused my world to collapse, leaving ...",Sadness,Joy
4,He was distraught by the horrors he saw during...,Sadness,Fear
...,...,...,...
116,Link’s quest to save Hyrule from calamity Gann...,Joy,Sadness
117,Though he was rejoicing that the stab wound lo...,Joy,Joy
118,I worry that someday I’ll forget the love I’ve...,Joy,Joy
119,She yelled urgently before I was able to leave...,Joy,Joy


In [ ]:
# Load Naive Bayes Classifiers
# We will sue Multinomial Naive Bayes for text classification
# reference: https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()

In [ ]:
X_train = Dictionary_Sen["Sentence"]
y_train = Dictionary_Sen["Label"]
# We didn't previous tokenize X_test, just tokenize it now
X_test = Dictionary_Sen_test["Sentence"]
y_test = Dictionary_Sen_test["Label"]

In [ ]:
# reference: https://scikit-learn.org/1.4/tutorial/text_analytics/working_with_text_data.html
# we cannot directly input sentence in the model
# We will use built in bag of word
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

# fit the model
clf.fit(X_train_counts, y_train)

MultinomialNB()

In [ ]:
# predict the labels for test data
X_test_counts = count_vect.transform(X_test)
predicted = clf.predict(X_test_counts)

In [ ]:
# create new confusion matrix
y_true = y_test
y_pred = predicted
confusion_matrix(y_true, y_pred, labels=labels)

array([[10,  0,  0,  1,  5, 10],
       [ 8,  2,  0,  1,  3,  4],
       [ 2,  1,  0,  1,  0,  4],
       [ 5,  1,  0,  5,  0,  6],
       [ 3,  0,  0,  0, 11, 12],
       [ 1,  0,  0,  0,  7, 18]])

In [ ]:
# We can convert confusion matrix to pandas dataframe for better readability
# Set labels as both column and index
confusion_matrix_df = pd.DataFrame(confusion_matrix(y_true, y_pred, labels=labels), index=labels, columns=labels)
confusion_matrix_df

,Fear,Anger,Surprise,Disgust,Sadness,Joy
Fear,10,0,0,1,5,10
Anger,8,2,0,1,3,4
Surprise,2,1,0,1,0,4
Disgust,5,1,0,5,0,6
Sadness,3,0,0,0,11,12
Joy,1,0,0,0,7,18


In [ ]:
# Get the performance of exist NB model on dataset
accuracy, precision, recall, f1_score = calculate_performance("Joy")
print("Accuracy =", accuracy)
print("Precision =", precision)
print("Recall =", recall)
print("F1-score =", f1_score)

Accuracy = 0.6363636363636364
Precision = 0.6923076923076923
Recall = 0.3333333333333333
F1-score = 0.45
